In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os
sys.path.append('..')

import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV

from get_data import get_data, get_splitted_data
from models.active_model import ActiveLearningExperiment
from models.strategy import *
from models.utils import ObstructedY

from misc.config import c

In [4]:
from sklearn.metrics import auc

In [6]:
a = np.abs(np.random.normal(size=100))

auc(np.arange(100), a)

86.269868694254072